<a href="https://colab.research.google.com/github/tyhrr/web-scrap/blob/main/Copia_de_proyecto_scrapping_tugurium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PRIMERA PARTE**




Para abordar esta tarea de scraping y extracción de un glosario en formato CSV desde el sitio web, se detalla paso a paso el proceso con todas las consideraciones técnicas. Trabajaremos con Python y la librería BeautifulSoup para extraer el contenido HTML, y utilizaremos Pandas para manipular los datos y generar el CSV.

Paso 1: Análisis del sitio web

El primer paso es analizar cómo están organizados los datos en el sitio web. En el caso de http://www.tugurium.com/gti/termino.php?Tr=cast, parece que cada término del glosario tiene una URL con un identificador único en el parámetro Tr.

Así que necesitamos entender:

- **Cómo se generan esos parámetros.**

- **Si hay una lista maestra de términos o si debes iterar a través de combinaciones posibles.**

- **Esto también incluye inspeccionar la estructura del HTML para identificar dónde se encuentran los términos y sus traducciones.**

**SEGUNDA PARTE**

Configuracion del entorno:

In [1]:
#Instalaremos las herramientas necesarias:
!pip install requests
!pip install beautifulsoup4
!pip install pandas

- **requests:** Para hacer las peticiones HTTP y descargar las páginas.

- **BeautifulSoup4:** Para procesar el HTML y extraer los datos.

- **Pandas:** Para almacenar los resultados en un CSV.

**TERCERA PARTE**

Primero, hacemos el scraping de una única página como prueba. Por ejemplo, vamos a extraer el término y la traducción de una URL específica texto en cursiva

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL para el término CAST
url = 'http://www.tugurium.com/gti/termino.php?Tr=CAST'

# Realizar la solicitud a la página
response = requests.get(url)
html = response.content

# Parsear el contenido HTML
soup = BeautifulSoup(html, 'html.parser')

# Encontrar todos los elementos div con clase 'bs-callout bs-callout-warning'
entradas = soup.find_all('div', class_='bs-callout bs-callout-warning')

resultados = []

for entrada in entradas:
    # Encontrar la sección con clase 'i_link' dentro de cada entrada
    seccion = entrada.find('section', class_='i_link')
    if seccion:
        # Encontrar todos los elementos de lista dentro de la sección
        items = seccion.find_all('li')
        for item in items:
            # Obtener el término (CAST)
            termino = "CAST"
            # Obtener la traducción del tag <a>
            traduccion = item.find('a').text.strip()
            # Obtener la información adicional (si existe) del tag <div>
            info_adicional = item.find('div').text.strip() if item.find('div') else ""
            # Guardar el resultado
            resultados.append([termino, traduccion, info_adicional])

# Convertir la lista de resultados en un DataFrame
df = pd.DataFrame(resultados, columns=['Término', 'Traducción', 'Información Adicional'])

# Guardar los resultados en un archivo CSV
df.to_csv('glosario.csv', index=False)

# Mostrar el DataFrame
print(df)


  Término                       Traducción      Información Adicional
0    CAST            Cable and Satelite TV                       CAST
1    CAST  Computer-Aided Software Testing                       CAST
2    CAST                  test automation  automatización de pruebas
3    CAST           symmetric cryptography     criptografía simétrica
4    CAST                          casting                   moldeado


**CUARTA PARTE**

Generalización para múltiples términos
Como podemos ver, el script no capta de manera correcta el termino junto con su definicion. Agregaremos ese feature mas adelante.
Una vez que el scraping para una página funciona, el siguiente paso es iterar a través de todas las páginas posibles del glosario.

Si se conocen los valores del parámetro Tr, puedes hacer una lista de esos valores y generar las URLs dinámicamente.
Si no conoces los valores y necesitas descubrirlos dinámicamente, puedes hacer scraping de una página que tenga un índice de términos (si existe), o puedes usar una técnica como generar combinaciones de letras (si los términos son identificados alfabéticamente).

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL para las entradas que comienzan con 'D'
url = 'http://www.tugurium.com/gti/contenido.php?INI=D'

# Realizar la solicitud a la página
response = requests.get(url)
html = response.content

# Parsear el contenido HTML
soup = BeautifulSoup(html, 'html.parser')

resultados = []

# Encontrar todas las secciones con clase 'i_link'
secciones = soup.find_all('section', class_='i_link')

for seccion in secciones:
    # Encontrar todos los elementos de lista dentro de la sección
    items = seccion.find_all('li')
    for item in items:
        # Obtener el término del enlace
        termino = item.find('a').text.strip()

        # Obtener la URL del término
        url_termino = item.find('a')['href']

        # Obtener la información adicional (si existe) del tag <div>
        info_adicional = item.find('div').text.strip() if item.find('div') else ""

        # Guardar el resultado
        resultados.append([termino, url_termino, info_adicional])

# Convertir la lista de resultados en un DataFrame
df = pd.DataFrame(resultados, columns=['Término', 'URL', 'Información Adicional'])

# Guardar los resultados en un archivo CSV
df.to_csv('glosario_letra_D.csv', index=False)

# Mostrar el DataFrame
print(df)
df.head(20)

                                        Término  \
0                                             D   
1                                       D layer   
2                                        D link   
3                                        D link   
4                                      D region   
...                                         ...   
1911                           dynamic web page   
1912  Dynamically Reconfigurable Robotic System   
1913                 dynamically typed language   
1914                                     dynamo   
1915                                    DynaTac   

                                                    URL  \
0                                      termino.php?Tr=D   
1                              termino.php?Tr=D%20layer   
2                               termino.php?Tr=D%20link   
3                               termino.php?Tr=D%20link   
4                             termino.php?Tr=D%20region   
...                              

,Término,URL,Información Adicional
0,D,termino.php?Tr=D,D
1,D layer,termino.php?Tr=D%20layer,capa D
2,D link,termino.php?Tr=D%20link,description link
3,D link,termino.php?Tr=D%20link,enlace D
4,D region,termino.php?Tr=D%20region,capa D
5,D-nn,termino.php?Tr=D-nn,D-nn
6,D-pad,termino.php?Tr=D-pad,directional pad
7,D-structure,termino.php?Tr=D-structure,estructura-D
8,D-sub,termino.php?Tr=D-sub,D-sub
9,D-subminiature,termino.php?Tr=D-subminiature,"D-subminiatura, D-sub"


Como podemos ver, el script anterior nos da como resultado una descripcion que contiene una URL, la cual nos conduce a otra pagina. Completaremos el script anterior para que navegue a esa pagina web, localice la descripcion/descripciones deseadas




In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def obtener_descripciones(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    descripciones = []
    for p in soup.select('div.definicion p'):
        texto = p.get_text(strip=True)
        if texto:
            descripciones.append(texto)
    return descripciones

# URL para las entradas que comienzan con 'D'
url = 'http://www.tugurium.com/gti/contenido.php?INI=D'

# Realizar la solicitud a la página
response = requests.get(url)
html = response.content

# Parsear el contenido HTML
soup = BeautifulSoup(html, 'html.parser')

resultados = []

# Encontrar todas las secciones con clase 'i_link'
secciones = soup.find_all('section', class_='i_link')

for seccion in secciones:
    # Encontrar todos los elementos de lista dentro de la sección
    items = seccion.find_all('li')
    for item in items:
        # Obtener el término del enlace
        termino = item.find('a').text.strip()

        # Obtener la URL del término
        url_termino = 'http://www.tugurium.com/gti/' + item.find('a')['href']

        # Obtener las descripciones del término
        descripciones = obtener_descripciones(url_termino)

        # Obtener la información adicional (si existe) del tag <div>
        info_adicional = item.find('div').text.strip() if item.find('div') else ""

        # Guardar el resultado
        resultados.append({
            'Término': termino,
            'Descripciones': descripciones,
            'Información Adicional': info_adicional
        })

# Convertir la lista de resultados en un DataFrame
df = pd.DataFrame(resultados)

# Guardar los resultados en un archivo CSV
df.to_csv('glosario_letra_D_con_descripciones.csv', index=False)

# Mostrar el DataFrame
print(df)
print(df.head(20))

                                        Término Descripciones  \
0                                             D            []   
1                                       D layer            []   
2                                        D link            []   
3                                        D link            []   
4                                      D region            []   
...                                         ...           ...   
1911                           dynamic web page            []   
1912  Dynamically Reconfigurable Robotic System            []   
1913                 dynamically typed language            []   
1914                                     dynamo            []   
1915                                    DynaTac            []   

            Información Adicional  
0                               D  
1                          capa D  
2                description link  
3                        enlace D  
4                          capa D  
...

In [3]:
df.head(20)

,Término,Traducción,Información Adicional
0,CAST,Cable and Satelite TV,CAST
1,CAST,Computer-Aided Software Testing,CAST
2,CAST,test automation,automatización de pruebas
3,CAST,symmetric cryptography,criptografía simétrica
4,CAST,casting,moldeado


He notado que la pagina de Tugurium tiene una especie de sistema anti script o de limites de solicitudes simultaneas, por lo que decidi implementar los siguientes cambios:

1. Reduce la concurrencia o la elimina por completo.
2. Aumenta los tiempos de espera entre solicitudes.
3. Usa un servicio de proxy para rotar las direcciones IP.
4. Contacta a los propietarios del sitio web para obtener permiso o acceso a una API si está disponible.

De esta manera logramos que el script sea mas veloz y realice un mayor numero de tareas en simultaneo.





In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures
import time
import random
from requests.exceptions import RequestException

def obtener_detalles(url, max_retries=3):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    for attempt in range(max_retries):
        try:
            time.sleep(random.uniform(1, 3))  # Espera aleatoria entre 1 y 3 segundos
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            detalle = soup.select_one('section.desc')
            if detalle:
                return detalle.text.strip()
            return ""
        except RequestException as e:
            print(f"Error al obtener {url}: {e}. Intento {attempt + 1} de {max_retries}")
            if attempt == max_retries - 1:
                print(f"No se pudo obtener {url} después de {max_retries} intentos")
                return ""
            time.sleep(2 ** attempt)  # Retroceso exponencial
    return ""

def procesar_entrada(item):
    try:
        termino = item.find('a').text.strip()
        descripcion_breve = item.find('div').text.strip() if item.find('div') else ""
        url_termino = 'http://www.tugurium.com/gti/' + item.find('a')['href']
        detalle_definicion = obtener_detalles(url_termino)
        return {
            'Término': termino,
            'Descripciones': descripcion_breve,
            'Información Adicional': detalle_definicion
        }
    except Exception as e:
        print(f"Error al procesar entrada: {e}")
        return None

# URL para las entradas que comienzan con 'D'
url = 'http://www.tugurium.com/gti/contenido.php?INI=D'

# Realizar la solicitud a la página
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar todas las entradas
items = soup.select('section.i_link li')

# Limitar a 200 entradas para este ejemplo
items = items[:200]

# Usar multithreading para procesar las entradas en paralelo
start_time = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    resultados = list(executor.map(procesar_entrada, items))

# Filtrar resultados None
resultados = [r for r in resultados if r is not None]

# Convertir la lista de resultados en un DataFrame
df = pd.DataFrame(resultados)

# Guardar los resultados en un archivo CSV
df.to_csv('glosario_letra_D.csv', index=False)

# Mostrar el DataFrame completo
print(df.to_string())

print(f"\nTiempo total de ejecución: {time.time() - start_time} segundos")
print(f"\nNúmero total de entradas procesadas: {len(df)}")

                                      Término                                                  Descripciones                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

Implemento las siguientes mejoras/modificaciones a mi script:

1. Un print que sea cada 10 o cada 100 términos scrapeados, del tipo (“scraped 100 out of 9,000 terms”). Da una idea de cuánto tiempo falta, de si sigue avanzando, etc.

2. Tratar de no repetir código: siendo que la cuarta parte ya guarda 'glosario_letra_D.csv', lo ideal sería que luego se lea ese archivo y tan solo se ejecute obtener_descripciones (en lugar de volver a generar el df con todos los términos).

3. Sería bueno que el executor, cada N items, lea un csv, concatene el df tal como está ahora en esta iteración (luego de excluir los items que ya estan en el csv) y sobreescriba el csv. Así guardas incrementalmente los resultados y si algo falla no tenes que empezar desde cero.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures
import time
import random
from requests.exceptions import RequestException
import os

def obtener_detalles(url, max_retries=3):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    for attempt in range(max_retries):
        try:
            time.sleep(random.uniform(1, 3))  # Espera aleatoria entre 1 y 3 segundos
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            detalle = soup.select_one('section.desc')
            if detalle:
                return detalle.text.strip()
            return ""
        except RequestException as e:
            print(f"Error al obtener {url}: {e}. Intento {attempt + 1} de {max_retries}")
            if attempt == max_retries - 1:
                print(f"No se pudo obtener {url} después de {max_retries} intentos")
                return ""
            time.sleep(2 ** attempt)  # Retroceso exponencial
    return ""

def procesar_entrada(item):
    try:
        termino = item.find('a').text.strip()
        descripcion_breve = item.find('div').text.strip() if item.find('div') else ""
        url_termino = 'http://www.tugurium.com/gti/' + item.find('a')['href']
        detalle_definicion = obtener_detalles(url_termino)
        return {
            'Término': termino,
            'Descripciones': descripcion_breve,
            'Información Adicional': detalle_definicion
        }
    except Exception as e:
        print(f"Error al procesar entrada: {e}")
        return None

def leer_csv_existente(nombre_archivo):
    if os.path.exists(nombre_archivo):
        return pd.read_csv(nombre_archivo)
    return pd.DataFrame(columns=['Término', 'Descripciones', 'Información Adicional'])

def guardar_csv_incremental(df, nombre_archivo):
    df_existente = leer_csv_existente(nombre_archivo)
    df_concatenado = pd.concat([df_existente, df]).drop_duplicates(subset='Término')
    df_concatenado.to_csv(nombre_archivo, index=False)

def obtener_terminos_nuevos(items, df_existente):
    terminos_existentes = df_existente['Término'].tolist()
    return [item for item in items if item.find('a').text.strip() not in terminos_existentes]

def scrapeo_progresivo(items, nombre_archivo, batch_size=100):
    df_existente = leer_csv_existente(nombre_archivo)
    terminos_nuevos = obtener_terminos_nuevos(items, df_existente)

    total_terminos = len(terminos_nuevos)
    print(f"Total de términos nuevos: {total_terminos}")

    resultados = []
    contador = 0
    start_time = time.time()

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        for resultado in executor.map(procesar_entrada, terminos_nuevos):
            if resultado:
                resultados.append(resultado)
                contador += 1
                if contador % batch_size == 0:
                    guardar_csv_incremental(pd.DataFrame(resultados), nombre_archivo)
                    resultados = []  # Limpiar la lista después de guardar
                    print(f"Scraped {contador} out of {total_terminos} terms")

    # Guardar cualquier resultado restante después de que termine el scraping
    if resultados:
        guardar_csv_incremental(pd.DataFrame(resultados), nombre_archivo)

    print(f"\nTiempo total de ejecución: {time.time() - start_time} segundos")
    print(f"\nNúmero total de entradas procesadas: {contador}")

# URL para las entradas que comienzan con 'D'
url = 'http://www.tugurium.com/gti/contenido.php?INI=D'

# Realizar la solicitud a la página
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Encontrar todas las entradas
items = soup.select('section.i_link li')

# Aqui podemos limitar las entradas de ser necesario para detener el script
items = items[:]

# Nombre del archivo CSV
nombre_archivo = 'glosario_letra_D.csv'

# Ejecutar el scrapeo progresivo
scrapeo_progresivo(items, nombre_archivo, batch_size=100)


Total de términos nuevos: 1715
Scraped 100 out of 1715 terms
Scraped 200 out of 1715 terms
Scraped 300 out of 1715 terms
Scraped 400 out of 1715 terms
Scraped 500 out of 1715 terms
Scraped 600 out of 1715 terms
Scraped 700 out of 1715 terms
Scraped 800 out of 1715 terms
Scraped 900 out of 1715 terms
Scraped 1000 out of 1715 terms


In [9]:
df = pd.read_csv('glosario_letra_D.csv')
df.head(100)

,Término,Descripciones,Información Adicional
0,D,D,Del mismo modo que el lenguaje de programación...
1,D layer,capa D,"Ver ""D region""."
2,D link,description link,enlace descriptivo
3,D region,capa D,Capa inferior de aire ionizado sobre la tierra...
4,D-nn,D-nn,Categoría de conectores con diferente número d...
...,...,...,...
95,data characteristic,característica de datos,"Rasgo, calidad o propiedad inherente de los da..."
96,data classification,clasificación de datos,Asignación de un nivel de sensibilidad a la in...
97,data collection,recolección de datos,Procedimiento por el cual se reunen documentos...
98,data communication,comunicación de datos,Proceso de intercambio de información entre di...
